In [1]:
import sys
import importlib
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt
import sys
import awkward as ak
import scipy
import ROOT
sys.path.insert(0,"../")


Welcome to JupyROOT 6.30/07


In [2]:
sys.path.insert(0,"../python/HNL_Plotting_HelperFunctions")
import MuonSystemReader
import ABCD_e
import Processing_Helpers
import analysis_helpers
import ABCD_Tau_noLepton_noJetVeto


/cvmfs/cms.cern.ch/el9_amd64_gcc12/lcg/root/6.30.07-024df6516c17fd2edef848a927a788f1/lib/ROOT/_facade.py:154: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  return _orig_ihook(name, *args, **kwds)
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, w

In [5]:
HNL_2GeV_10ctau_path = "root://cmseos.fnal.gov//store/group/lpclonglived/amalbert/HNL_Tau_Search/analyzer_update_111825/HNL_tau_mN_2_ctau_1000/normalized/HNL_tau_mN_2_ctau_1000.root"
signal_events  = MuonSystemReader.loadTree_nanoFactory(HNL_2GeV_10ctau_path)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/dask_awkward/lib/structure.py:911: UserWarning: Please ensure that dask.awkward<nCscRechitClusters, npartitions=4>
        is partitionwise-compatible with dask.awkward<getitem, npartitions=4>
        (e.g. counts comes from a dak.num(array, axis=1)),
        otherwise this unflatten operation will fail when computed!
  warnings.warn(


In [6]:
signal_xSec = 1
processed_lumi = 109080

In [7]:
generated_signalEvents = ak.count(signal_events.evtNum)
signal_normalization_factor = (processed_lumi*signal_xSec/generated_signalEvents).compute()
print(signal_normalization_factor)

0.11104550544640131


In [8]:
sizeCut=160
phiCut=2

### Check contamination at 10 pb (rough UL from Run 2 search), no additional DNN/StationVeto Selections

In [ ]:
clusterSize_allCutsDNN0, dPhi_allCutsDNN0 = ABCD_e.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'lepLooseId', failID = 'lepMediumId', blind=False, DNN_cut=0)
clusterSize_allCutsDNN0MT, dPhi_allCutsDNN0MT = ABCD_e.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'lepMediumId', failID = 'lepTightId', blind=False, DNN_cut=0)
clusterSize_allCutsDNN0SR, dPhi_allCutsDNN0SR = ABCD_e.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'lepTightId', failID = None, blind=False, DNN_cut=0)

/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/python3/3.9.14-c10287ae9cadff55334e60003302c349/lib/python3.9/contextlib.py:126: UserWarning: Creating scratch directories is taking a surprisingly long time. (1.91s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)
/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Oth

In [ ]:
print(f"signal contamination VR1 no DNN/StatonVeto {ak.count_nonzero(clusterSize_allCutsDNN0)*signal_normalization_factor}")
print(f"signal contamination VR2 no DNN/StatonVeto {ak.count_nonzero(clusterSize_allCutsDNN0MT)*signal_normalization_factor}")
print(f"signal in SR {ak.count_nonzero(clusterSize_allCutsDNN0SR)*signal_normalization_factor}")

print("in bin A")
print(f"bin A VR1: {ak.count_nonzero((clusterSize_allCutsDNN0>160) & (dPhi_allCutsDNN0>2))*signal_normalization_factor}")
print(f"bin A VR2: {ak.count_nonzero((clusterSize_allCutsDNN0MT>160) & (dPhi_allCutsDNN0MT>2))*signal_normalization_factor}")
print(f"bin A SR: {ak.count_nonzero((clusterSize_allCutsDNN0SR>160) & (dPhi_allCutsDNN0SR>2))*signal_normalization_factor}")  

signal contamination VR1 no DNN/StatonVeto 38.86592690624045
signal contamination VR2 no DNN/StatonVeto 43.30774712409651
signal in SR 128.81278631782553
in bin A
bin A VR1: 24.430011198208287
bin A VR2: 25.5404662526723
bin A SR: 66.62730326784079


### Signal Contamination with StationVeto Applied

In [9]:
clusterSize_allCutsStationVeto, dPhi_allCutsStationVeto = ABCD_FirstStudy_Helper.return_clusterSize_dPhiClusterTau_allSelections(signal_events, passID = 'tauIsLoose', failID = 'tauIsMedium', blind=False)
clusterSize_allCutsStationVetoMT, dPhi_allCutsStationVetoMT = ABCD_FirstStudy_Helper.return_clusterSize_dPhiClusterTau_allSelections(signal_events, passID = 'tauIsMedium', failID = 'tauIsTight', blind=False)
clusterSize_allCutsStationVetoSR, dPhi_allCutsStationVetoSR = ABCD_FirstStudy_Helper.return_clusterSize_dPhiClusterTau_allSelections(signal_events, passID = 'tauIsTight', failID = None, blind=False)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])

In [11]:
print(f"signal contamination VR1 with Station12Veto {ak.count_nonzero(clusterSize_allCutsStationVeto)*signal_normalization_factor}")
print(f"signal contamination VR2 with Station12Veto {ak.count_nonzero(clusterSize_allCutsStationVetoMT)*signal_normalization_factor}")
print(f"signal in SR with Station12Veto {ak.count_nonzero(clusterSize_allCutsStationVetoSR)*signal_normalization_factor}")

print("in bin A")
print(f"bin A VR1: {ak.count_nonzero((clusterSize_allCutsStationVeto>160) & (dPhi_allCutsStationVeto>2))*signal_normalization_factor}")
print(f"bin A VR2: {ak.count_nonzero((clusterSize_allCutsStationVetoMT>160) & (dPhi_allCutsStationVetoMT>2))*signal_normalization_factor}")
print(f"bin A SR: {ak.count_nonzero((clusterSize_allCutsStationVetoSR>160) & (dPhi_allCutsStationVetoSR>2))*signal_normalization_factor}")  

signal contamination VR1 with Station12Veto 2.220910108928026
signal contamination VR2 with Station12Veto 7.7731853812480916
signal in SR with Station12Veto 42.197292069632496
in bin A
bin A VR1: 1.110455054464013
bin A VR2: 5.552275272320065
bin A SR: 29.98228647052835


### Signal Contamination with DNN>0.99975

In [13]:
clusterSize_allCutsDNN, dPhi_allCutsDNN = ABCD_FirstStudy_Helper.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'tauIsLoose', failID = 'tauIsMedium', blind=False, DNN_cut=0.99975)
clusterSize_allCutsDNNMT, dPhi_allCutsDNNMT = ABCD_FirstStudy_Helper.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'tauIsMedium', failID = 'tauIsTight', blind=False, DNN_cut=0.99975)
clusterSize_allCutsDNNSR, dPhi_allCutsDNNSR = ABCD_FirstStudy_Helper.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'tauIsTight', failID = None, blind=False, DNN_cut=0.99975)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34109 instead
  warnings.warn(
/uscms/home/amalbert/.local/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40989 instead
  warnings.warn(
/uscms/home/amalbert/.local/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37637 instead
  warnings.warn(
/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
   

In [14]:
print(f"signal contamination VR1 with DNN {ak.count_nonzero(clusterSize_allCutsDNN)*signal_normalization_factor}")
print(f"signal contamination VR2 with DNN {ak.count_nonzero(clusterSize_allCutsDNNMT)*signal_normalization_factor}")
print(f"signal in SR with DNN {ak.count_nonzero(clusterSize_allCutsDNNSR)*signal_normalization_factor}")

print("in bin A")
print(f"bin A VR1: {ak.count_nonzero((clusterSize_allCutsDNN>160) & (dPhi_allCutsDNN>2))*signal_normalization_factor}")
print(f"bin A VR2: {ak.count_nonzero((clusterSize_allCutsDNNMT>160) & (dPhi_allCutsDNNMT>2))*signal_normalization_factor}")
print(f"bin A SR: {ak.count_nonzero((clusterSize_allCutsDNNSR>160) & (dPhi_allCutsDNNSR>2))*signal_normalization_factor}")

signal contamination VR1 with DNN 3.3313651633920394
signal contamination VR2 with DNN 7.7731853812480916
signal in SR with DNN 15.546370762496183
in bin A
bin A VR1: 2.220910108928026
bin A VR2: 4.441820217856052
bin A SR: 5.552275272320065


### Testing

In [9]:
clusterSize_allCutsDNN, dPhi_allCutsDNN = ABCD_Tau_noLepton_noJetVeto.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'tauIsLoose', failID = 'tauIsMedium', blind=False, DNN_cut=0.99975)
clusterSize_allCutsDNNMT, dPhi_allCutsDNNMT = ABCD_Tau_noLepton_noJetVeto.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'tauIsMedium', failID = 'tauIsTight', blind=False, DNN_cut=0.99975)
clusterSize_allCutsDNNSR, dPhi_allCutsDNNSR = ABCD_Tau_noLepton_noJetVeto.return_clusterSize_dPhiClusterTau_allSelectionsDNN(signal_events, passID = 'tauIsTight', failID = None, blind=False, DNN_cut=0.99975)

/uscms/home/amalbert/.local/lib/python3.9/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector
/cvmfs/cms.cern.ch/el8_amd64_gcc12/external/py3-numpy/1.24.3-0831476b9e4ddf0b4b9eb1f4c971c0d1/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])

In [11]:
print(f"signal contamination VR1 with DNN {ak.count_nonzero(clusterSize_allCutsDNN)*signal_normalization_factor}")
print(f"signal contamination VR2 with DNN {ak.count_nonzero(clusterSize_allCutsDNNMT)*signal_normalization_factor}")
print(f"signal in SR with DNN {ak.count_nonzero(clusterSize_allCutsDNNSR)*signal_normalization_factor}")

print("in bin A")
print(f"bin A VR1: {ak.count_nonzero((clusterSize_allCutsDNN<160) & (dPhi_allCutsDNN<2))*signal_normalization_factor}")
print(f"bin A VR2: {ak.count_nonzero((clusterSize_allCutsDNNMT<160) & (dPhi_allCutsDNNMT<2))*signal_normalization_factor}")
print(f"bin A SR: {ak.count_nonzero((clusterSize_allCutsDNNSR<160) & (dPhi_allCutsDNNSR<2))*signal_normalization_factor}")

print("in bin B")
print(f"bin B VR1: {ak.count_nonzero((clusterSize_allCutsDNN>160) & (dPhi_allCutsDNN<2))*signal_normalization_factor}")
print(f"bin B VR2: {ak.count_nonzero((clusterSize_allCutsDNNMT>160) & (dPhi_allCutsDNNMT<2))*signal_normalization_factor}")
print(f"bin B SR: {ak.count_nonzero((clusterSize_allCutsDNNSR>160) & (dPhi_allCutsDNNSR<2))*signal_normalization_factor}")

print("in bin C")
print(f"bin C VR1: {ak.count_nonzero((clusterSize_allCutsDNN<160) & (dPhi_allCutsDNN>2))*signal_normalization_factor}")
print(f"bin C VR2: {ak.count_nonzero((clusterSize_allCutsDNNMT<160) & (dPhi_allCutsDNNMT>2))*signal_normalization_factor}")
print(f"bin C SR: {ak.count_nonzero((clusterSize_allCutsDNNSR<160) & (dPhi_allCutsDNNSR>2))*signal_normalization_factor}")

print("in bin D")
print(f"bin D VR1: {ak.count_nonzero((clusterSize_allCutsDNN>160) & (dPhi_allCutsDNN>2))*signal_normalization_factor}")
print(f"bin D VR2: {ak.count_nonzero((clusterSize_allCutsDNNMT>160) & (dPhi_allCutsDNNMT>2))*signal_normalization_factor}")
print(f"bin D SR: {ak.count_nonzero((clusterSize_allCutsDNNSR>160) & (dPhi_allCutsDNNSR>2))*signal_normalization_factor}")

signal contamination VR1 with DNN 0.33313651633920394
signal contamination VR2 with DNN 0.7773185381248091
signal in SR with DNN 1.110455054464013
in bin A
bin A VR1: 0.0
bin A VR2: 0.0
bin A SR: 0.11104550544640131
in bin B
bin B VR1: 0.0
bin B VR2: 0.33313651633920394
bin B SR: 0.6662730326784079
in bin C
bin C VR1: 0.11104550544640131
bin C VR2: 0.0
bin C SR: 0.0
in bin D
bin D VR1: 0.22209101089280261
bin D VR2: 0.44418202178560523
bin D SR: 0.33313651633920394
